In [1]:
from time import process_time
import statistics
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras.backend as K
from sklearn.metrics import accuracy_score
from datetime import datetime
import pandas as pd

In [2]:
import signal

class timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

In [3]:
default_value=tf.keras.backend.floatx()

### Inputs

In [22]:
layers_types=['float32','float64']
inputs_types=['int8','int16','int32','float16','float32','float64']
method_name='Format des inputs/layers'
model_name='CNN'

In [24]:
result=pd.DataFrame(columns=['Modèle','Nb(paramètres)','Date','Méthode','Paramètres','CPU + Sys time','Précision'])
for inputs_type in inputs_types:
    for layers_type in layers_types:
        time,accuracy,date,parameters,nb_params=GetTime(model_name,inputs_type,layers_type,iteration=100,time_out=120)
        result=result.append({'Modèle':model_name,'CPU + Sys time':time,'Précision':accuracy,'Date':date,'Méthode':method_name,'Paramètres':parameters,'Nb(paramètres)':nb_params}, ignore_index=True)
        print('Modèle: ',model_name,'Input: ',inputs_type, '&', 'Layers: ',layers_type,'--> time : ',time,'--> accuracy : ',accuracy)

Modèle:  CNN Input:  int8 & Layers:  float32 --> time :  timeout --> accuracy :  0.0984
Modèle:  CNN Input:  int8 & Layers:  float64 --> time :  timeout --> accuracy :  0.0908
Modèle:  CNN Input:  int16 & Layers:  float32 --> time :  timeout --> accuracy :  0.1893
Modèle:  CNN Input:  int16 & Layers:  float64 --> time :  timeout --> accuracy :  0.1986
Modèle:  CNN Input:  int32 & Layers:  float32 --> time :  timeout --> accuracy :  0.1554
Modèle:  CNN Input:  int32 & Layers:  float64 --> time :  timeout --> accuracy :  0.1786
Modèle:  CNN Input:  float16 & Layers:  float32 --> time :  timeout --> accuracy :  0.2817
Modèle:  CNN Input:  float16 & Layers:  float64 --> time :  timeout --> accuracy :  0.1631
Modèle:  CNN Input:  float32 & Layers:  float32 --> time :  timeout --> accuracy :  0.2593
Modèle:  CNN Input:  float32 & Layers:  float64 --> time :  timeout --> accuracy :  0.1079
Modèle:  CNN Input:  float64 & Layers:  float32 --> time :  9.037169096709986 --> accuracy :  0.1042
Mod

### Fill the table

In [34]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))

x_train = x_train.astype("float32")
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32")
x_test = np.reshape(x_test, (-1, 28, 28, 1))

In [35]:
layers_type='float32'
tf.keras.backend.set_floatx(layers_type)

In [46]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same",dtype='layers_type'),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same",dtype='layers_type'),
        tf.keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same",dtype='layers_type'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10,dtype='layers_type'),
    ]
)

In [47]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
model.fit(x_train, y_train, epochs=3)
model.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.7728 - sparse_categorical_accuracy: 0.9248
Epoch 2/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1479 - sparse_categorical_accuracy: 0.9628
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - loss: 0.1185 - sparse_categorical_accuracy: 0.9673


[0.11853490024805069, 0.9672999978065491]

In [44]:
y_pred=model.predict(x_test)

In [45]:
accuracy_score(np.argmax(y_pred,1),y_test)

0.97

In [ ]:
y_pred=model.predict(x_test)

accuracys.append(accuracy_score(np.argmax(y_pred,1),y_test))

In [31]:
model.evaluate(x_train,x_test)

ValueError: Data cardinality is ambiguous:
  x sizes: 60000
  y sizes: 10000
Please provide data which shares the same first dimension.

In [20]:
def GetModel(layers_type=None,model_name='MLP'):
    tf.keras.backend.set_floatx(layers_type)
    if model_name=='MLP':
        model = tf.keras.models.Sequential([
              tf.keras.layers.Flatten(input_shape=(28,28,1)),
              tf.keras.layers.Dense(64,activation='relu', dtype=layers_type),
              tf.keras.layers.Dense(64,activation='relu', dtype=layers_type),
              tf.keras.layers.Dense(10,activation='softmax', dtype=layers_type)
            ])
    elif model_name=='CNN':
        model = tf.keras.Sequential([
        tf.keras.Input(shape=(28, 28, 1),dtype=layers_type),
        tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same",dtype=layers_type),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same",dtype=layers_type),
        tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same",dtype=layers_type),
        tf.keras.layers.Flatten(dtype=layers_type),
        tf.keras.layers.Dense(10,dtype=layers_type)])
    else:
        print('This model doesn\'t exist.')
    return model 

def GetParametersNumber(model):
    trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])
    return trainable_count+non_trainable_count

def GetTime(model_name,inputs_type,layers_type,iteration,time_out):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = np.reshape(x_train, (-1, 28, 28, 1))
    x_test = np.reshape(x_test, (-1, 28, 28, 1))
    
    x_train_type = x_train.astype(inputs_type)
    x_test_type = x_test.astype(inputs_type)
    
    model=GetModel(layers_type=layers_type,model_name=model_name)

    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=tf.keras.optimizers.RMSprop(),
                 metrics=['accuracy'])

    model.fit(x_train, y_train,
                 batch_size=1000,
                 epochs=1,
                 validation_split=0.2,
                 verbose=0)
    
    nb_params=GetParametersNumber(model)
    date=datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    parameters={'inputs_type':inputs_type,'layers_type':layers_type}
    temps=[]
    accuracys=[]
    with timeout(seconds=time_out):
        try:
            for k in range(iteration):
                start=process_time()
                y_pred=model.predict(x_test_type)
                stop=process_time()
                temps.append(stop-start)
                accuracys.append(accuracy_score(np.argmax(y_pred,1),y_test))
                tf.keras.backend.set_floatx(default_value)
            return statistics.mean(temps),statistics.mean(accuracys),date,parameters,nb_params
        except Exception as e:
            tf.keras.backend.set_floatx(default_value)
            return 'timeout', statistics.mean(accuracys),date,parameters,nb_params